# Churn Rate Analysis

### Install packages
Installing packages and dependents to query from bigquery and loading bigquery package

In [ ]:
##bigquery installation
!pip install google-cloud-bigquery-storage --quiet 
##bigquery uses pandas internally, pyarrow is required to enable this
!pip install pyarrow --quiet 
##for progress bar
!pip install tqdm --quiet
## jupyter notebook ui
!pip install ipywidgets --quiet 

In [ ]:
## Load magic string extension from google.cloud.bigquery to be enable magic cell for bigquery in my notebook.

%load_ext google.cloud.bigquery

## Assumptions of the data
- Every tuple is referencing 1 customer
- Account_length is in months
- Charges are per year
- If they do not have a voicemail plan they cannot have any voicemails
- If they do not have an international plan they can still make international calls (at a cost)

## Churn rate overview
The churn rate is the annual percentage rate that customers cancel their service, calculated as the number of customers who churned divided by the total number of customers, times by 100 to get a percentage.

First the churn rate and total_customers (sample size) is calcualted. This helps understand the baseline.

Then basic analysis (mean, median, mode, standard deviation) is done to get a better understanding of the key data points.

Finally segementation and analysis of the groups is conducted, and location specific analysis is done.



In [ ]:
%%bigquery
SELECT 
    COUNT(DISTINCT unique_id) AS total_customer,
    SUM(churn_binary) AS churned_customer
FROM `nomadic-ocean-395807.churn_rate.customer_data`;


In [ ]:
%%bigquery
SELECT 
    SUM(churn_binary)/COUNT(area_code)*100 AS churn_rate
FROM `nomadic-ocean-395807.churn_rate.customer_data`;


## Mean, Median, Mode and Standard Deviation
For basic analysis, a churned comparison of the mean, median, mode and standard deviation for different variables is done.

### Number of Voicemail messages
Higher mean number of voicemail messages for active accounts
Median is 0 for churned for and acitve accounts


In [ ]:
%%bigquery
SELECT 
    churn,
    AVG(number_vmail_messages) AS mean,
    APPROX_QUANTILES(number_vmail_messages,100)[OFFSET(50)] AS median,
    APPROX_TOP_COUNT(number_vmail_messages,1) AS mode,
    STDDEV(number_vmail_messages) AS stddev
FROM `nomadic-ocean-395807.churn_rate.customer_data`
GROUP BY churn;


### Total International calls
Comparable mean number of total international calls, very slightly higher for active accounts
Median is 4 for both churned and active accounts

In [ ]:
%%bigquery
SELECT 
    churn,
    AVG(total_intl_calls) AS mean, 
    APPROX_QUANTILES(total_intl_calls,100)[OFFSET(50)] AS median,
    APPROX_TOP_COUNT(total_intl_calls,1) AS mode,
    STDDEV(total_intl_charge) AS stddev
FROM `nomadic-ocean-395807.churn_rate.customer_data`
GROUP BY churn;


### Number of Customer service calls
Higher mean number of customer service calls for churned accounts
Median is 1 call higher for churned accounts

In [ ]:
%%bigquery
SELECT 
    churn,
    AVG(number_customer_service_calls) AS mean, 
    APPROX_QUANTILES(number_customer_service_calls,100)[OFFSET(50)] AS median,
    APPROX_TOP_COUNT(number_customer_service_calls,1) AS mode,
    STDDEV(number_customer_service_calls) AS stddev
FROM `nomadic-ocean-395807.churn_rate.customer_data`
GROUP BY churn;


### Account length
Slightly higher mean account length for churned accounts
Median is 2 months higher for churned accounts

In [ ]:
%%bigquery
SELECT 
    churn,
    AVG(account_length) AS mean,
    APPROX_QUANTILES(account_length,100)[OFFSET(50)] AS median,
    APPROX_TOP_COUNT(account_length,1) AS mode,
    STDDEV(account_length) AS stddev
FROM `nomadic-ocean-395807.churn_rate.customer_data`
GROUP BY churn;


### Total charges
Higher mean total charges for churned accounts
Median is $8.15 higher for churned accounts

In [ ]:
%%bigquery
SELECT
    churn,
    AVG(total_charges) AS mean,
    APPROX_QUANTILES(total_charges,100)[OFFSET(50)] AS median,
    APPROX_TOP_COUNT(total_charges,1) AS mode,
    STDDEV(total_charges) AS stddev
FROM `nomadic-ocean-395807.churn_rate.customer_data`
GROUP BY churn;


## Segmenting
Segmenting the customers into different cohorts can make it easier to identify the type of customers at risk of churn.
The following variables have been segmented; plan type (voicemail and international), account length, yrr and number of customer service calls
A churn analysis was run to compare the segments within the each variable.

### Account length
The cohorts with the most amount of churns are 4-8 and 8-12 years, they also have the most amount of active accounts so this is to be expected. As a percentage, the longest tenure accounts 16-20 years has a higher churn rate then the other groups. The 0-4 and 4-8 years shows the lowest percentages also.

In [ ]:
%%bigquery
SELECT 
    SUM(churn_binary) AS churn_count,
    COUNT(
        CASE 
            WHEN NOT churn THEN 1 
        END) AS active_count,
        account_length_group_years
FROM `nomadic-ocean-395807.churn_rate.customer_data`
GROUP BY account_length_group_years
ORDER BY churn_count DESC;


In [ ]:
%%bigquery
SELECT
    ROUND((SUM(churn_binary)/COUNT(unique_id)*100),2) AS churn_rate,
    account_length_group_years
FROM `nomadic-ocean-395807.churn_rate.customer_data`
GROUP BY account_length_group_years
ORDER BY churn_rate DESC;


### YRR
The cohort with the most amount of churns is $60.00 - $79.99 $40.00 - $59.99, they also have the most amount of active accounts so this is to be expected. As a percentage, the cohort $80.00 - $99.99 has a much higher churn rate then the other groups. 

In [ ]:
%%bigquery
SELECT 
    SUM(churn_binary) AS churn_count,
    COUNT(
        CASE 
            WHEN NOT churn THEN 1 
        END) AS active_count,
    yrr_group
FROM `nomadic-ocean-395807.churn_rate.customer_data`
GROUP BY yrr_group
ORDER BY churn_count DESC;


In [ ]:
%%bigquery
SELECT 
    ROUND((SUM(churn_binary)/COUNT(unique_id)*100),2) AS churn_rate,
    yrr_group
FROM `nomadic-ocean-395807.churn_rate.customer_data`
GROUP BY yrr_group
ORDER BY churn_rate DESC;


### Voicemail plan
Number of accounts with VM messages (and therefore VM plan) grouped by churn.


In [ ]:
%%bigquery
SELECT 
    churn,
    SUM(IF (number_vmail_messages > 0, 1, 0)) AS voicemail_count,
FROM 
    `nomadic-ocean-395807.churn_rate.customer_data`
GROUP BY 
    voice_mail_plan,
    churn
HAVING voice_mail_plan IS TRUE
ORDER BY voicemail_count;


#### churn rate - Voicemail
Churn rate comparison between has voicemail plan and does not have voicemail plan.
This can help identify if having voicemail plan increases churn rate

In [ ]:
%%bigquery
SELECT 
    voice_mail_plan,
    ROUND((SUM(churn_binary)/COUNT(unique_id)*100),2) AS churn_rate
FROM `nomadic-ocean-395807.churn_rate.customer_data`
GROUP BY voice_mail_plan;


### International plan
Number of accounts with International plan grouped by churn and not churned.

In [ ]:
%%bigquery
SELECT 
    churn,
    COUNT(
        CASE 
            WHEN NOT churn THEN 1 
        END) AS active_count,
    COUNT(
        CASE 
            WHEN churn THEN 1 
        END) AS churn_count,
    COUNT(
        CASE 
            WHEN international_plan THEN 1 
        END) AS international_plan
FROM `nomadic-ocean-395807.churn_rate.customer_data`
GROUP BY churn;


#### Churn rate - International plan
Churn rate comparison between has international plan and does not have international plan.
This can help identify if having international plan increases churn rate.

In [ ]:
%%bigquery
SELECT
    international_plan,
    ROUND((SUM(churn_binary)/COUNT(unique_id)*100),2) AS churn_rate
FROM `nomadic-ocean-395807.churn_rate.customer_data`
GROUP BY international_plan;


#### International plan costs and churn rate
International plan and international calls and charges comparison.

In [ ]:
%%bigquery
SELECT
    churn,
    international_plan,
    SUM(total_intl_charge) AS intl_charge, 
    SUM(total_intl_calls) AS intl_calls, 
    SUM(total_intl_minutes) AS intl_minutes
FROM `nomadic-ocean-395807.churn_rate.customer_data`
GROUP BY 
    churn, 
    international_plan;


In [ ]:
%%bigquery
SELECT 
    ROUND((SUM(churn_binary)/COUNT(unique_id)*100),2) AS churn_rate,
    state
FROM `nomadic-ocean-395807.churn_rate.customer_data`
GROUP BY state
ORDER BY churn_rate DESC;


In [ ]:
%%bigquery
SELECT 
    state,
    COUNT(state) AS account_count,
    ROUND((SUM(churn_binary)/COUNT(unique_id)*100),2) AS churn_rate,
    SUM(churn_binary) AS churn_count, 
    SUM(number_vmail_messages) AS vmail_count,
    SUM(total_charges) AS charges_count, 
    SUM(number_customer_service_calls) AS cs_calls_count
FROM `nomadic-ocean-395807.churn_rate.customer_data`
GROUP BY state
ORDER BY churn_rate DESC;


### Number of Customer service calls
There is a signifcant increase in churn rate when customer service calls are 4 and above

In [ ]:
%%bigquery
SELECT 
    number_customer_service_calls,
    COUNT (number_customer_service_calls) AS call_count,
    ROUND((SUM(churn_binary)/COUNT(unique_id)*100),2) AS churn_rate
FROM `nomadic-ocean-395807.churn_rate.customer_data`
GROUP BY number_customer_service_calls 
ORDER BY churn_rate DESC;


## Location data
### State
Location specific data is evaluated. This can help individual branches with marketing decisions and tailor retention process on a state by state basis

In [ ]:
%%bigquery
SELECT 
    state,
    COUNT(state) AS account_count,
    ROUND((SUM(churn_binary)/COUNT(unique_id)*100),2) AS churn_rate,
    SUM(churn_binary) AS churn_count, 
    SUM(number_vmail_messages) AS vmail_count,
    SUM(total_charges) AS charges_count, 
    SUM(number_customer_service_calls) AS cs_calls_count
FROM `nomadic-ocean-395807.churn_rate.customer_data`
GROUP BY state
ORDER BY churn_rate DESC;
